# Get Up and Running Quickly

## 🌍 Overview

This quickstart aims to give you a small illustration of what ZenML can do. We will:

- Import some data from a public dataset (Adult Census Income), then train two models (SGD and Random Forest)
- Compare and evaluate which model performs better, and deploy the best one.
- Run a prediction on the deployed model.

Along the way we will also show you how to:

- Automatically version, track, and cache data, models, and other artifacts,
- Track model hyperparameters and metrics in an experiment tracking tool,

This will give you enough to get started building your own ZenML Pipelines.
Let's dive in!


DIAGRAM SHOWING THE FLOW


- introduction to what the quickstart is about
- what will be covered here / what we'll do

## Run on Colab

You can use Google Colab to see ZenML in action, no signup / installation
required!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/zenml-io/zenml/blob/main/examples/quickstart/new_quickstart/new_quickstart.ipynb)

# 1. Install Requirements

Let's install ZenML to get started. First we'll install the latest version of
ZenML as well as the two integrations we'll need for this quickstart: `sklearn`
and `mlflow`.

In [ ]:
#TODO: add things relating to cloudflare pipelines etc and zenml installation
!pip install -q -e "../../../.[dev]"

In [ ]:
# !zenml integration install sklearn mlflow -y
!zenml init
# %pip install pyngrok pyparsing==2.4.2  # required for Colab #TODO: STILL NEEDED?

# # automatically restart kernel
# import IPython
# IPython.Application.instance().kernel.do_shutdown(restart=True)

Please wait for the installation to complete before running subsequent cells. At the end of the installation, the notebook kernel will automatically restart.

# 2. Import Data

We'll start off by importing our data. In this quickstart we'll be working with
[the Adult Census Income](https://archive.ics.uci.edu/dataset/2/adult) dataset
which is publicly available on the UCI Machine Learning Repository. The task is
to predict whether a person makes over $50k a year based on a number of
features. These features are things like age, work class, education level,
marital status, occupation, relationship, race, sex, capital gain, capital loss,
hours per week, and native country.

When you're getting started with a machine learning problem you'll want to do
something similar to this: import your data and get it in the right shape for
your training. ZenML mostly gets out of your way when you're writing your Python
code, as you'll see from the following cell.

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split

from zenml import step
from zenml.steps import Output


@step
def training_data_loader() -> (
    Output(
        X_train=pd.DataFrame,
        X_test=pd.DataFrame,
        y_train=pd.Series,
        y_test=pd.Series,
    )
):
    """Load the Census Income dataset as tuple of Pandas DataFrame / Series."""
    # Load the dataset
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    column_names = [
        "age",
        "workclass",
        "fnlwgt",
        "education",
        "education-num",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "capital-gain",
        "capital-loss",
        "hours-per-week",
        "native-country",
        "income",
    ]
    data = pd.read_csv(
        url, names=column_names, na_values="?", skipinitialspace=True
    )

    # Drop rows with missing values
    data = data.dropna()

    # Encode categorical features and drop original columns
    categorical_cols = [
        "workclass",
        "education",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "native-country",
    ]
    data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

    # Encode target feature
    data["income"] = data["income"].apply(
        lambda x: 1 if x.strip() == ">50K" else 0
    )

    # Separate features and target
    X = data.drop("income", axis=1)
    y = data["income"]

    # Split the dataset into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    return X_train, X_test, y_train, y_test

We download the data, dropping some columns and then splitting it up into train
and test sets. The whole function is decorated with the `@step` decorator, which
tells ZenML to track this function as a step in the pipeline. This means that
ZenML will automatically version, track, and cache the data that is produced by
this function. This is a very powerful feature, as it means that you can
reproduce your data at any point in the future, even if the original data source
changes or disappears.

You'll also notice that we have included type hints for the inputs and outputs
to the function. These are not only useful for anyone reading your code, but
help ZenML process your data in a way appropriate to the specific data types.

Sometimes a step will have multiple outputs. In order to give each output a
unique name, we also use the `Output()` annotation.

ZenML is built in a way that allows you to experiment with your data and build
your pipelines as you work, so if you want to call this function to see how it
works, you can just call it directly. Here we take a look at the first few rows
of your training dataset.

In [35]:
X_train, X_test, y_train, y_test = training_data_loader()
X_train.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
19863,53,168539,5,0,0,70,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
24342,49,56841,13,0,0,70,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
10027,28,154571,10,0,0,40,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
25710,60,188236,6,0,0,40,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
13824,53,87158,9,0,0,40,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False


Everything looks as we'd expect and the values are all in the right format. We
can shift to training some models now! 🥳

# 3. Train Models

Now that we have our data it makes sense to train some models to get a sense of
how difficult the task is. The Census Income
dataset is sufficiently large and complex that it's unlikely we'll be able to
train a model that behaves perfectly since the problem is inherently complex,
but we can get a sense of what a reasonable baseline looks like.

We'll start with two simple models, a SGD Classifier and a Random Forest
Classifier, both batteries-included from `sklearn`. We'll train them both on the
same data and then compare their performance.

Since we're starting our work properly, it makes sense to start tracking the
experimentation that we're doing. ZenML integrates with MLflow to make this
easy. This happens out of the box when using our experiment tracker integration
and stack components. We'll see how this works below, but first let's set up
ZenML to know that it should use the MLFlow experiment tracker.

In [36]:
# Register the MLflow experiment tracker
!zenml experiment-tracker register mlflow_tracker --flavor=mlflow

# Register a new stack with our experiment tracker
!zenml stack register quickstart -a default\
                                 -o default\
                                 -e mlflow_tracker

!zenml stack set quickstart

Connected to the ZenML server: 'http://127.0.0.1:8237'
Running with active workspace: 'default' (repository)
Running with active stack: 'default' (repository)
⠋ Registering experiment tracker 'mlflow_tracker'...
⠙ Registering experiment tracker 'mlflow_tracker'...
⠙ Registering experiment tracker 'mlflow_tracker'...

╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /Users/strickvl/.pyenv/versions/3.10.11/envs/quickstart/bin/zenml:8 in       │
│ <module>                                                                     │
│                                                                              │
│   5 from zenml.cli.cli import cli                                            │
│   6 if __name__ == '__main__':                                               │
│   7 │   sys.argv[0] = re.sub(r'(-script\.pyw|\.exe)?$', '', sys.argv[0])     │
│ ❱ 8 │   sys.exit(cli())                                                      │
│   9                            

At this point our stack looks like this. We can now write the steps where we'll
train our models, making sure to specify the name of our experiment tracker in
the `@step` decorator.

In [37]:
import mlflow

from sklearn.base import ClassifierMixin
from sklearn.ensemble import RandomForestClassifier

from zenml.client import Client

experiment_tracker = Client().active_stack.experiment_tracker


@step(experiment_tracker=experiment_tracker.name)
def random_forest_trainer_mlflow(
    X_train: pd.DataFrame,
    y_train: pd.Series,
) -> ClassifierMixin:
    """Train a sklearn Random Forest classifier and log to MLflow."""
    mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
    model = RandomForestClassifier()
    model.fit(X_train.to_numpy(), y_train.to_numpy())
    train_acc = model.score(X_train.to_numpy(), y_train.to_numpy())
    print(f"Train accuracy: {train_acc}")
    return model


from sklearn.linear_model import SGDClassifier


@step(experiment_tracker=experiment_tracker.name)
def sgd_trainer_mlflow(
    X_train: pd.DataFrame,
    y_train: pd.Series,
) -> ClassifierMixin:
    """Train a SGD classifier and log to MLflow."""
    mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
    model = SGDClassifier()
    model.fit(X_train.to_numpy(), y_train.to_numpy())
    train_acc = model.score(X_train.to_numpy(), y_train.to_numpy())
    print(f"Train accuracy: {train_acc}")
    return model

Reloading configuration file /Users/strickvl/coding/zenml/repos/zenml/examples/quickstart/new_quickstart/.zen/config.yaml


The end goal of this quick baseline evaluation is to understand which of the two
models performs better. We'll use the `evaluator` step to compare the two
models. This step takes in the two models we trained above, and compares them on
the test data we created earlier. It returns whichever model performs best. Note
that we aren't using an `Output` annotation here, since there is a single output
coming from this step.

In [38]:
@step
def evaluator(
    X_test: pd.DataFrame,
    y_test: pd.Series,
    model1: ClassifierMixin,
    model2: ClassifierMixin,
) -> ClassifierMixin:
    """Calculate the accuracy on the test set and return the best model of two."""
    test_acc1 = model1.score(X_test.to_numpy(), y_test.to_numpy())
    test_acc2 = model2.score(X_test.to_numpy(), y_test.to_numpy())
    print(f"Test accuracy ({model1.__class__.__name__}): {test_acc1}")
    print(f"Test accuracy ({model2.__class__.__name__}): {test_acc2}")
    return model1 if test_acc1 > test_acc2 else model2

We'll likely want to use our model in the future so instead of simply outputting
the model we'll use the MLflow model registry to store it. This allows us to
version the model for retrieval and use later on as well as to use other
functionality made possible within the MLflow dashboard. This step is a bit
different from the ones listed above in that we're using a pre-built ZenML step
instead of just writing our own. You'll often come across these pre-built steps
for common workflows.

In [39]:
from zenml.integrations.mlflow.steps.mlflow_registry import (
    mlflow_register_model_step,
)

model_name = "zenml-quickstart-model"

register_model = mlflow_register_model_step.with_options(
    parameters=dict(
        name=model_name,
        description="The first run of the Quickstart pipeline.",
    )
)

We're now at the point where can bring all these steps together into a single
pipeline, the top-level organising entity in ZenML. Creating such a pipeline is
as simple as adding a `@pipeline` decorator to a function. This specific
pipeline doesn't return a value, but that option is available to you if you need.

In [40]:
from zenml import pipeline


@pipeline
def train_and_register_model_pipeline() -> None:
    """Train a model."""
    register_model.after(evaluator)

    X_train, X_test, y_train, y_test = training_data_loader()
    model1 = random_forest_trainer_mlflow(X_train=X_train, y_train=y_train)
    model2 = sgd_trainer_mlflow(X_train=X_train, y_train=y_train)
    best_model = evaluator(
        X_test=X_test, y_test=y_test, model1=model1, model2=model2
    )
    register_model(best_model)

We've used the built-in MLflow registry to store our model, but ZenML doesn't
yet know that we want to use the MLflow flavor of the model registry stack
component in our stack. Let's add that now and update our stack.

In [41]:
# Register the MLflow model registry
!zenml model-registry register mlflow_registry --flavor=mlflow

# Register a new stack with the new stack components
!zenml stack update quickstart -r mlflow_registry

Connected to the ZenML server: 'http://127.0.0.1:8237'
Running with active workspace: 'default' (repository)
Running with active stack: 'quickstart' (repository)
⠋ Registering model registry 'mlflow_registry'...
⠙ Registering model registry 'mlflow_registry'...

╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /Users/strickvl/.pyenv/versions/3.10.11/envs/quickstart/bin/zenml:8 in       │
│ <module>                                                                     │
│                                                                              │
│   5 from zenml.cli.cli import cli                                            │
│   6 if __name__ == '__main__':                                               │
│   7 │   sys.argv[0] = re.sub(r'(-script\.pyw|\.exe)?$', '', sys.argv[0])     │
│ ❱ 8 │   sys.exit(cli())                                                      │
│   9                                                                          │
│       

DIAGRAM SHOWING THE NEW STACK

We're ready to run the pipeline now, which we can do just -- as with the step -- by calling the
pipeline function itself.

In [42]:
train_and_register_model_pipeline()

Reloading configuration file /Users/strickvl/coding/zenml/repos/zenml/examples/quickstart/new_quickstart/.zen/config.yaml
Registered pipeline train_and_register_model_pipeline (version 4).
Running pipeline train_and_register_model_pipeline on stack quickstart (caching enabled)
Step training_data_loader has started.
Step training_data_loader has finished in 13.231s.
Step random_forest_trainer_mlflow has started.


2023/06/28 17:31:43 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


Train accuracy: 1.0
Step random_forest_trainer_mlflow has finished in 50.517s.
Step sgd_trainer_mlflow has started.


2023/06/28 17:32:35 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


Train accuracy: 0.7653031621700029
Step sgd_trainer_mlflow has finished in 23.595s.
Step evaluator has started.
Test accuracy (RandomForestClassifier): 0.8491629371788496
Test accuracy (SGDClassifier): 0.7576661694016243
Step evaluator has finished in 3.290s.
Step mlflow_register_model_step has started.
MLflow model registry does not take a version as an argument. Registering a new version for the model 'zenml-quickstart-model' a version will be assigned automatically.


2023/06/28 17:33:10 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: zenml-quickstart-model, version 5


Registered model zenml-quickstart-model with version 5 from source file:///Users/strickvl/Library/Application Support/zenml/local_stores/35fcee21-53e6-4de0-a49f-d238ea1d5040/mlruns/729179721969064096/04a2d7d6d4134955b88fba322c043d50/artifacts/model.
Step mlflow_register_model_step has finished in 2.784s.
Pipeline run train_and_register_model_pipeline-2023_06_28-15_31_25_192331 has finished in 1m47s.
Dashboard URL: http://127.0.0.1:8237/workspaces/default/pipelines/af180b4f-1155-41b4-b807-089f76962f36/runs


MAYBE SHOW THE ZENML DASHBOARD HERE

ALSO MAYBE SHOW THE MLFLOW UI + HOW TO ACCESS IT HERE

In [ ]:
# Set up Cloudflare tunnel
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb

!zenml up --port 8237 & cloudflared tunnel --url http://localhost:8237

[[[And you can use `wget -qO- http://localhost:55555/quicktunnel` to get the
magic hostname.]]]

TODO: Reimplement this natively under the hood

We're using MLflow for our experiment tracking. If you'd like to inspect the
MLflow dashboard to see your experiments and what's been logged so far, run the
following cell. This cell will spin up a local server that you can access via
the link mentioned after the "Listening at:" `INFO` log statement.

In [46]:
# TODO: Add cloudflare tunnel for MLflow UI

In [43]:
import os
from zenml.integrations.mlflow.mlflow_utils import get_tracking_uri

os.environ["MLFLOW_TRACKING_URI"] = get_tracking_uri()
!mlflow ui --backend-store-uri $MLFLOW_TRACKING_URI

[2023-06-28 17:33:38 +0200] [40251] [INFO] Starting gunicorn 20.1.0
[2023-06-28 17:33:38 +0200] [40251] [INFO] Listening at: http://127.0.0.1:5000 (40251)
[2023-06-28 17:33:38 +0200] [40251] [INFO] Using worker: sync
[2023-06-28 17:33:38 +0200] [40252] [INFO] Booting worker with pid: 40252
[2023-06-28 17:33:38 +0200] [40253] [INFO] Booting worker with pid: 40253
[2023-06-28 17:33:38 +0200] [40254] [INFO] Booting worker with pid: 40254
[2023-06-28 17:33:39 +0200] [40255] [INFO] Booting worker with pid: 40255
^C
[2023-06-28 17:33:52 +0200] [40251] [INFO] Handling signal: int
[2023-06-28 17:33:52 +0200] [40254] [INFO] Worker exiting (pid: 40254)
[2023-06-28 17:33:52 +0200] [40255] [INFO] Worker exiting (pid: 40255)
[2023-06-28 17:33:52 +0200] [40252] [INFO] Worker exiting (pid: 40252)
[2023-06-28 17:33:52 +0200] [40253] [INFO] Worker exiting (pid: 40253)


Talk about the pipeline output

Explain why we need the most recent model version

In [11]:
from zenml.client import Client

most_recentmodel_version_number = (
    Client()
    .active_stack.model_registry.list_model_versions(metadata={})[0]
    .version
)
most_recentmodel_version_number

'4'

Now we've trained our model, and we've found the best one, we want to deploy it and run some inference on the deployed model

In [12]:
from zenml.integrations.mlflow.steps.mlflow_deployer import (
    mlflow_model_registry_deployer_step,
)
from zenml.integrations.mlflow.steps.mlflow_registry import (
    mlflow_register_model_step,
)
from zenml.model_registries.base_model_registry import (
    ModelRegistryModelMetadata,
)

model_deployer = mlflow_model_registry_deployer_step.with_options(
    parameters=dict(
        registry_model_name=model_name,
        registry_model_version=most_recentmodel_version_number,
    )
)

Something about services + why we're doing it that way

In [21]:
from zenml.services import BaseService
from zenml.client import Client


@step(enable_cache=False)
def prediction_service_loader() -> BaseService:
    """Load the model service of our train_and_register_model_pipeline."""
    client = Client()
    model_deployer = client.active_stack.model_deployer
    services = model_deployer.find_model_server(
        pipeline_name="train_and_register_model_pipeline",
        running=True,
    )
    return services[0]


@step
def predictor(
    service: BaseService,
    data: pd.DataFrame,
) -> Output(predictions=list):
    """Run a inference request against a prediction service."""
    service.start(timeout=10)  # should be a NOP if already started
    prediction = service.predict(data.to_numpy())
    # prediction = prediction.argmax(axis=-1)
    print(f"Prediction is: {prediction.tolist()}")
    return prediction.tolist()


In [ ]:
# Register the MLflow model deployer
!zenml model-deployer register mlflow_deployer --flavor=mlflow

# Register a new stack with the new stack components
!zenml stack update quickstart -d mlflow_deployer

Explain our new pipeline

In [22]:
@pipeline
def register_and_deploy_model() -> None:
    """Print the name of the model."""
    prediction_service_loader.after(model_deployer)
    predictor.after(prediction_service_loader)
    model_deployer()
    _, inference_data, _, _ = training_data_loader()
    model_deployment_service = prediction_service_loader()
    predictor(service=model_deployment_service, data=inference_data)

In [23]:
register_and_deploy_model()

Registered pipeline register_and_deploy_model (version 6).
Running pipeline register_and_deploy_model on stack quickstart (caching enabled)
Step mlflow_model_registry_deployer_step has started.


Updating an existing MLflow deployment service: MLFlowDeploymentService[ac19964c-358a-4aeb-84d3-ba6f27194822] (type: model-serving, flavor: mlflow)


Output()

MLflow deployment service started and reachable at:
    http://127.0.0.1:8000/invocations

Step mlflow_model_registry_deployer_step has finished in 13.161s.
Step training_data_loader has started.
Step training_data_loader has finished in 3.590s.
Step prediction_service_loader has started.
Step prediction_service_loader has finished in 0.211s.
Step predictor has started.


Prediction is: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
register_and_deploy_model.model.last_successful_run.steps['predictor'].output.load()

In [ ]:
!zenml model-registry models list

In [ ]:
!zenml model-registry models list-versions zenml-quickstart-model

In [ ]:
!zenml model-deployer models list

In [ ]:
!zenml model-deployer models describe "ac19964c-358a-4aeb-84d3-ba6f27194822"